In [1]:
!pip install fasttext

     |████████████████████████████████| 71kB 5.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3095314 sha256=e32da5277a344fa5cf3ac8617e39c16fbd8e0a6e85c311814ebb7a0cbde8fe64
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [5]:
from google.colab import files
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,GRU,Embedding,Flatten
from tensorflow.keras.models import Model
import keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import fasttext.util

In [6]:
data=pd.read_csv('/content/prepared_data.csv')
data.head()

,SMS_TEXT,ENGLISH_TEXT
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [7]:
data['SMS_LEN'] = data['SMS_TEXT'].str.split().apply(len)
data = data[data['SMS_LEN'] < 39]



data['english_len'] = data['ENGLISH_TEXT'].str.split().apply(len)
data = data[data['english_len'] < 40]

data['ENGLISH_INPUT'] = '<start> ' + data['ENGLISH_TEXT'].astype(str)
data['ENGLISH_OUTPUT'] = data['ENGLISH_TEXT'].astype(str) + ' <end>'

data = data.drop(['ENGLISH_TEXT','SMS_LEN','english_len'], axis=1)
# only for the first sentance add a toke <end> so that we will have <end> in tokenizer

print(data.shape)
data.iloc[0]['ENGLISH_INPUT']=str(data.iloc[0]['ENGLISH_INPUT'])+' <end>'
data.iloc[0]['ENGLISH_OUTPUT']=str(data.iloc[0]['ENGLISH_OUTPUT'])+' <end>'

(1988, 3)


In [8]:
from sklearn.model_selection import train_test_split
train_data,test_data= train_test_split(data,test_size=0.01, random_state=42)
print(train_data.shape)
print(test_data.shape)

(1968, 3)
(20, 3)


In [9]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n',char_level=False,lower=False,oov_token=True)
print("SMS_TEXT")
tokenizer.fit_on_texts(train_data['SMS_TEXT'].values)
print("English text")
tokenizer_e = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n',char_level=False,lower=False,oov_token=True)
tokenizer_e.fit_on_texts(train_data['ENGLISH_INPUT'].values)

SMS_TEXT
English text


In [10]:
encoder_vocabluary=tokenizer.word_index
decoder_vocabluary=tokenizer_e.word_index
print(len(encoder_vocabluary))
print(len(decoder_vocabluary))

4606
3597


In [3]:
fasttext.util.download_model('en', if_exists='ignore')

'cc.en.300.bin'

In [4]:
ft = fasttext.load_model('cc.en.300.bin')

In [43]:
embedding_matrix=np.zeros(((len(encoder_vocabluary)+1,300)))
for word,i in encoder_vocabluary.items():
  
    if type(word)==bool:
        continue
    else:
        embedding_vector=ft.get_word_vector(word)
        if embedding_vector is not None:
           embedding_matrix[i]=embedding_vector
print(embedding_matrix.shape)   
        


(4607, 300)


In [44]:
from joblib import dump, load
dump(embedding_matrix, 'encoder_embedding.joblib') 

['encoder_embedding.joblib']

In [46]:
decoder_embedding_matrix=np.zeros(((len(decoder_vocabluary)+1,300)))
for word,i in decoder_vocabluary.items():
  
    if type(word)==bool:
        continue
    else:
        embedding_vector=ft.get_word_vector(word)
        if embedding_vector is not None:
           decoder_embedding_matrix[i]=embedding_vector
print(decoder_embedding_matrix.shape)      

(3598, 300)


In [47]:
from joblib import dump, load
dump(decoder_embedding_matrix, 'decoder_embedding.joblib') 

['decoder_embedding.joblib']